<a href="https://colab.research.google.com/github/zaku2590/classGCI/blob/main/comp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# モジュールのインポート
import numpy as np  # 数値計算や配列操作を行うためのライブラリ
import pandas as pd  # 表形式のデータを扱うためのライブラリ
import matplotlib.pyplot as plt  # データ可視化のための基本的なグラフ描画ライブラリ
import seaborn as sns  # 高機能な統計グラフを描画するライブラリ
from sklearn.preprocessing import LabelEncoder  # カテゴリ変数を数値に変換するエンコーダ
from sklearn.ensemble import RandomForestClassifier  # ランダムフォレストによる分類器
from sklearn.linear_model import LogisticRegression # ロジスティック回帰
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold  # 層化K分割交差検証を行うクラス
from sklearn.metrics import roc_auc_score  # ROC AUCスコアを計算する評価指標

In [ ]:
PATH = '/content/'

train = pd.read_csv(PATH + 'train.csv')
test = pd.read_csv(PATH + 'test.csv')


,Id,Year,Age,School,Height,Weight,Sprint_40yd,Vertical_Jump,Bench_Press_Reps,Broad_Jump,Agility_3cone,Shuttle,Player_Type,Position_Type,Position,Drafted
0,0,2011,21.0,Lehigh,1.9050,140.160042,5.39,59.69,29.0,251.46,7.91,4.94,offense,offensive_lineman,OG,1.0
1,1,2011,24.0,Abilene Christian,1.8288,87.089735,4.31,101.60,16.0,332.74,NaN,NaN,offense,backs_receivers,WR,1.0
2,2,2018,21.0,Colorado St.,1.8542,92.986436,4.51,91.44,10.0,309.88,6.95,4.37,offense,backs_receivers,WR,1.0
3,3,2010,21.0,East Carolina,1.9304,148.778297,5.09,76.20,39.0,254.00,8.12,4.71,defense,defensive_lineman,DT,1.0
4,4,2016,21.0,California,1.8796,92.079251,4.64,78.74,NaN,281.94,7.13,4.20,offense,backs_receivers,WR,1.0


In [ ]:
# 使わない列の削除
train = train.drop(columns=["Id", "School"])
test = test.drop(columns=["Id","School"])

# 平均で補完する対象の列
cols_to_fill = ['Age', 'Sprint_40yd', 'Vertical_Jump', 'Bench_Press_Reps',
                'Broad_Jump', 'Agility_3cone', 'Shuttle']

# positionTypeで平均を埋める
for col in cols_to_fill:
    train[col + "_was_missing"] = train[col].isnull().astype(int)
    test[col + "_was_missing"] = test[col].isnull().astype(int)

    group_mean = train.groupby("Position_Type")[col].mean()
    train[col] = train[col].fillna(train["Position_Type"].map(group_mean))
    test[col] = test[col].fillna(test["Position_Type"].map(group_mean))


# カテゴリデータをラベルエンコーディング
target_cols = ["Player_Type", "Position_Type", "Position"]

for col in target_cols:
    # trainデータで平均Draft率を計算
    target_mean = train.groupby(col)["Drafted"].mean()

    # 新しいエンコード列名（例：Player_Type_TE）
    new_col = col + "_TE"

    # train, test にmap（目的変数との平均を特徴にする）
    train[new_col] = train[col].map(target_mean)
    test[new_col] = test[col].map(target_mean)

    # 元のカテゴリ列を削除
    train = train.drop(columns=[col])
    test = test.drop(columns=[col])

In [ ]:
for df in [train, test]:
    df['BMI'] = df['Weight'] / (df['Height'] ** 2)

# train = train.drop(columns=["Weight", "Height"])
# test = test.drop(columns=["Weight","Height"])

train.head()

,Year,Age,Height,Weight,Sprint_40yd,Vertical_Jump,Bench_Press_Reps,Broad_Jump,Agility_3cone,Shuttle,Drafted,Player_Type_TE,Position_Type_TE,Position_TE,BMI
0,2011,21.0,1.9050,140.160042,5.39,59.69,29.000000,251.46,7.910000,4.940000,1.0,0.63045,0.664368,0.642384,38.621956
1,2011,24.0,1.8288,87.089735,4.31,101.60,16.000000,332.74,7.230447,4.399422,1.0,0.63045,0.615842,0.594937,26.039614
2,2018,21.0,1.8542,92.986436,4.51,91.44,10.000000,309.88,6.950000,4.370000,1.0,0.63045,0.615842,0.594937,27.046212
3,2010,21.0,1.9304,148.778297,5.09,76.20,39.000000,254.00,8.120000,4.710000,1.0,0.69863,0.708440,0.715000,39.925004
4,2016,21.0,1.8796,92.079251,4.64,78.74,20.236408,281.94,7.130000,4.200000,1.0,0.63045,0.615842,0.594937,26.063390


In [ ]:
# 特徴量と目的変数に分ける
X = train.drop(columns=["Drafted"])
y = train["Drafted"]

models = {
    "RandomForest": RandomForestClassifier(n_estimators=100, max_depth=7, random_state=2025),
    "LightGBM": LGBMClassifier(random_state=2025)
}

# クロスバリデーション設定
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# モデルごとのスコア・予測格納用辞書
model_auc_dict = {}
model_test_pred_dict = {}

for name, model in models.items():
    print(f"\n=== Model: {name} ===")
    auc_scores = []
    test_pred_proba_list = []

    for fold, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
        print(f" Fold {fold + 1}")

        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        model.fit(X_train, y_train)
        y_valid_pred_proba = model.predict_proba(X_valid)[:, 1]

        auc = roc_auc_score(y_valid, y_valid_pred_proba)
        auc_scores.append(auc)
        print(f"  AUC: {round(auc, 4)}")

        test_pred_proba = model.predict_proba(test)[:, 1]
        test_pred_proba_list.append(test_pred_proba)

    mean_auc = np.mean(auc_scores)
    print(f"→ Average AUC for {name}: {round(mean_auc, 4)}")

    model_auc_dict[name] = mean_auc
    model_test_pred_dict[name] = np.mean(test_pred_proba_list, axis=0)

# AUC比較結果を表示
print("\n=== Model Comparison ===")
for name, auc in model_auc_dict.items():
    print(f"{name}: AUC = {round(auc, 4)}")


=== Model: RandomForest ===
 Fold 1
  AUC: 0.7929
 Fold 2
  AUC: 0.8424
 Fold 3
  AUC: 0.8386
 Fold 4
  AUC: 0.7867
 Fold 5
  AUC: 0.8288
→ Average AUC for RandomForest: 0.8179

=== Model: LightGBM ===
 Fold 1
[LightGBM] [Info] Number of positive: 1442, number of negative: 782
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000151 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1033
[LightGBM] [Info] Number of data points in the train set: 2224, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.648381 -> initscore=0.611932
[LightGBM] [Info] Start training from score 0.611932
  AUC: 0.8001
 Fold 2
[LightGBM] [Info] Number of positive: 1442, number of negative: 783
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000327 seconds.
You can set `force_col_wise=true` to remove

In [ ]:
submission_template = pd.read_csv(PATH + 'sample_submission.csv')

for name, test_pred in model_test_pred_dict.items():
    submission = submission_template.copy()
    submission["Drafted"] = test_pred
    submission.to_csv(PATH + f"{name.lower()}_submission.csv", index=False)